# OCD‑EEG Pipeline Notebook
End‑to‑end training **and** evaluation

##  Paths & config

In [3]:
from pathlib import Path
import logging, json, joblib, pprint

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

DATA_DIR = Path('/Users/ilyamikheev/Downloads/ML') # must contain subfolders per class
OUT_DIR  = Path('output')  # notebook output
OUT_DIR.mkdir(exist_ok=True)

CFG = {
    'split': {'test_size': 0.2, 'val_size': 0.1, 'random_state': 42},
    'preprocess': {},
    'nn': {},           # see trainers/nn_trainer for options
    'ml': {'tune': True, 'search': 'random', 'n_iter': 25, 'tune_lgbm': True, 'n_trials': 40},
}

## 1. Load dataset & stratified subject split

In [4]:
from ocd_classification.data.data_loader import load_dataset, split_dataset

raw_data = load_dataset(DATA_DIR, mode='train')
train_d, val_d, test_d = split_dataset(raw_data, **CFG['split'])

print('Subjects: ', {k: len(v['subject_ids']) for k,v in [('train',train_d),('val',val_d),('test',test_d)]})

/Users/ilyamikheev/PycharmProjects/ocd_classification/src/ocd_classification/data/data_loader.py:48: RuntimeWarning: No coordinate information found for channels ['empty']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr, preload=True, verbose=False)


Subjects:  {'train': 2, 'val': 2, 'test': 2}


/Users/ilyamikheev/PycharmProjects/ocd_classification/src/ocd_classification/data/data_loader.py:48: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['empty']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr, preload=True, verbose=False)


## 2. Preprocess & normalize (train / val)

In [5]:
from ocd_classification.preprocessing.preprocess import preprocess_data, normalize_data

X_tr, y_tr = preprocess_data(train_d, CFG['preprocess'], mode='train')[:2]
X_val, y_val = preprocess_data(val_d, CFG['preprocess'], mode='test')[:2]

X_tr, X_val = normalize_data(X_tr, X_val)
print(X_tr.shape, X_val.shape)

Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/R  1'), np.str_('Stimulus/R  3'), np.str_('Stimulus/R  4'), np.str_('Stimulus/R  7'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 11'), np.str_('Stimulus/S 12'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 15'), np.str_('Stimulus/S 16'), np.str_('Stimulus/S 20'), np.str_('Stimulus/S 21'), np.str_('Stimulus/S 22'), np.str_('Stimulus/S 23'), np.str_('Stimulus/S 24'), np.str_('Stimulus/S 25'), np.str_('Stimulus/S 30'), np.str_('Stimulus/S 31'), np.str_('Stimulus/S 33'), np.str_('Stimulus/S 34'), np.str_('Stimulus/S 35'), np.str_('Stimulus/S51'), np.str_('Stimulus/S52'), np.str_('Stimulus/S53'), np.str_('Stimulus/Start'), np.str_('UserDefined/Blink')]
    Using multitaper spectrum estimation with 7 DPSS windows
Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/R  1'), np.str_('Stimulus/R  3'), np.str_('Stimulus/R  4'), np.str_('Stimulus/S  1'), np.str_('Stimulus

## 3  Train models

In [6]:
from ocd_classification.trainers.nn_trainer import train_nn, save_model as save_torch
from ocd_classification.trainers.ml_trainer import train_ml

# NN
nn_model = train_nn(X_tr, y_tr, X_val, y_val, cfg=CFG['nn'])
save_torch(nn_model, OUT_DIR / 'nn_model.pt')

# ML
ml_models = train_ml(X_tr, y_tr, X_val, y_val, cfg=CFG['ml'])
with open(OUT_DIR / 'val_metrics.json', 'w') as f:
    json.dump(ml_models.get('metrics', {}), f, indent=2)
for name, m in ml_models.items():
    if name == 'metrics': continue
    joblib.dump(m, OUT_DIR / f'{name}.joblib')

/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-20 17:00:06,748 INFO: E00  loss 0.2442/3.5083  acc 0.952/0.369
2025-05-20 17:00:06,937 INFO: E01  loss 0.0008/9.6528  acc 1.000/0.369
2025-05-20 17:00:07,124 INFO: E02  loss 0.0002/21.4680  acc 1.000/0.371
2025-05-20 17:00:07,318 INFO: E03  loss 0.0001/22.9459  acc 1.000/0.373
2025-05-20 17:00:07,515 INFO: E04  loss 0.0000/22.1967  acc 1.000/0.373
2025-05-20 17:00:07,734 INFO: E05  loss 0.0000/21.4437  acc 1.000/0.373
2025-05-20 17:00:07,954 INFO: E06  loss 0.0000/21.4421  acc 1.000/0.373
2025-05-20 17:00:08,170 INFO: E07  loss 0.0000/21.4396  acc 1.000/0.373
2025-05-20 17:00:08,391 INFO: E08  loss 0.0000/20.3102  acc 1.000/0.373
2025-05-20 17:00:08,610 INFO: 

[LightGBM] [Info] Number of positive: 264, number of negative: 283
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 389590
[LightGBM] [Info] Number of data points in the train set: 547, number of used features: 2418
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.482633 -> initscore=-0.069498
[LightGBM] [Info] Start training from score -0.069498
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
2025-05-20 17:00:09,204 INFO: logreg val acc: 0.3666
2025-05-20 17:00:09,207 INFO: random_forest val acc: 0.3731
2025-05-20 17:00:09,245 INFO: svm val acc: 0.4273
/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

## 4  Preprocess & normalise (test)

In [7]:
X_test, y_test = preprocess_data(test_d, CFG['preprocess'], mode='test')[:2]
X_test, = normalize_data(X_test)

Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/R  1'), np.str_('Stimulus/R  3'), np.str_('Stimulus/R  4'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 11'), np.str_('Stimulus/S 12'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 15'), np.str_('Stimulus/S 16'), np.str_('Stimulus/S 17'), np.str_('Stimulus/S 20'), np.str_('Stimulus/S 21'), np.str_('Stimulus/S 22'), np.str_('Stimulus/S 24'), np.str_('Stimulus/S 25'), np.str_('Stimulus/S 30'), np.str_('Stimulus/S 31'), np.str_('Stimulus/S 34'), np.str_('Stimulus/S 35'), np.str_('Stimulus/S41'), np.str_('Stimulus/S51'), np.str_('Stimulus/S52'), np.str_('Stimulus/S53'), np.str_('Stimulus/Start'), np.str_('UserDefined/Blink')]
    Using multitaper spectrum estimation with 7 DPSS windows
Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/R  1'), np.str_('Stimulus/R  3'), np.str_('Stimulus/R  4'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S 10'), np.str_('Stimulus/

## 5  Evaluate all models on test

In [8]:
from ocd_classification.trainers.nn_trainer import load_model as load_nn
from ocd_classification.evaluate.evaluate import evaluate, save_results

models = {'nn': load_nn(OUT_DIR / 'nn_model.pt', input_shape=X_test.shape[1:])}
for p in OUT_DIR.glob('*.joblib'):
    models[p.stem] = joblib.load(p)

results = {k: evaluate(m, X_test, y_test) for k,m in models.items()}
pprint.pp(results)

save_results(results, OUT_DIR / "test_metrics.json")

2025-05-20 17:00:11,686 INFO: Loaded model ← output/nn_model.pt


{'nn': {'accuracy': 0.6746203904555315,
        'precision': 0.528169014084507,
        'recall': 0.9036144578313253,
        'f1': 0.6666666666666666,
        'roc_auc': np.float64(0.8529405758627732),
        'confusion_matrix': array([[161, 134],
       [ 16, 150]]),
        'true_negatives': 161,
        'false_positives': 134,
        'false_negatives': 16,
        'true_positives': 150},
 'svm': {'accuracy': 0.7982646420824295,
         'precision': 0.8288288288288288,
         'recall': 0.5542168674698795,
         'f1': 0.6642599277978339,
         'roc_auc': np.float64(0.880314478251991),
         'confusion_matrix': array([[276,  19],
       [ 74,  92]]),
         'true_negatives': 276,
         'false_positives': 19,
         'false_negatives': 74,
         'true_positives': 92},
 'logreg': {'accuracy': 0.7223427331887202,
            'precision': 0.5811965811965812,
            'recall': 0.8192771084337349,
            'f1': 0.68,
            'roc_auc': np.float64(0.8423933

/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/ilyamikheev/Library/Caches/pypoetry/virtualenvs/ocd-classification-2Pw0B5RN-py3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
